# Julia is fast

Very often, benchmarks are used to compare languages. These benchmarks can lead to long discussions, first as to exactly what is being benchmarked and secondly what explains the differences. These simple questions can sometimes get more complicated than you at first might imagine.

The purpose of this notebook is for you to see a simple benchmark for yourself. One can read the notebook and see what happened on the author's Macbook Pro with a 4-core Intel Core I7, or run the notebook yourself.

(This material began life as a wonderful lecture by Steven Johnson at MIT: https://github.com/stevengj/18S096/blob/master/lectures/lecture1/Boxes-and-registers.ipynb.)

## Outline of this notebook
1. Define the sum function
2. Implementations & benchmarking of sum in...
    - C (hand-written)
    - C (hand-written with -ffast-math)
    - python (built-in)
    - python (numpy)
    - python (hand-written)
    - Julia (built-in)
    - Julia (hand-written)
    - Julia (hand-written with SIMD)
3. Summary of benchmarks

---------

## ``sum``: An easy enough function to understand
Consider the sum function ``sum(a)``, which computes

\begin{align}
\text{sum}(a) = \sum_{i=1}^{n} a_i
\end{align}

where $n$ is the length of ``a``.

In [1]:
a = rand(10^7) # 1D vector of random numbers, uniform on [0,1)

10000000-element Vector{Float64}:
 0.477876464305932
 0.02989418273057276
 0.6486725185201311
 0.9289383013405196
 0.04451780891056478
 0.5085049320604675
 0.5267346474235319
 0.9137169621951242
 0.9946600719744176
 0.0046616782291516445
 0.5320084468484367
 0.5581614120180962
 0.22462983325674257
 ⋮
 0.20321066809688726
 0.14912522227579084
 0.41288606981759624
 0.3558253882496436
 0.7527946727995034
 0.34276927283741243
 0.005234897228630575
 0.2574180945183465
 0.14774388648608372
 0.7840888261792005
 0.28250057636038794
 0.775295760786879

In [2]:
sum(a)

4.999141791088371e6

The expected result is $0.5 \times 10^7$, since the mean of each entry is $0.5$

## Benchmarking a few ways in a few languages

In [3]:
@time sum(a)


  0.027860 seconds (1 allocation: 16 bytes)


4.999141791088371e6

In [4]:
@time sum(a)


  0.027417 seconds (1 allocation: 16 bytes)


4.999141791088371e6

In [5]:
@time sum(a)

  0.028334 seconds (1 allocation: 16 bytes)


4.999141791088371e6

The ``@time`` macro can yield noisy results, so it's not our best choice for benchmarking!

Luckily, Julia has a ``BenchmarkTools.jl`` package to make benchmarking easy and accurate:

In [6]:
using Pkg
Pkg.add("BenchmarkTools")

    Updating registry at `C:\Users\Joshu\.julia\registries\General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Resolving package versions...
  No Changes to `C:\Users\Joshu\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\Joshu\.julia\environments\v1.8\Manifest.toml`


In [7]:
using BenchmarkTools  

In [12]:
≈  # alias for the `isapprox` function

isapprox (generic function with 9 methods)

In [13]:
?isapprox

search: isapprox



```
isapprox(x, y; atol::Real=0, rtol::Real=atol>0 ? 0 : √eps, nans::Bool=false[, norm::Function])
```

Inexact equality comparison. Two numbers compare equal if their relative distance *or* their absolute distance is within tolerance bounds: `isapprox` returns `true` if `norm(x-y) <= max(atol, rtol*max(norm(x), norm(y)))`. The default `atol` is zero and the default `rtol` depends on the types of `x` and `y`. The keyword argument `nans` determines whether or not NaN values are considered equal (defaults to false).

For real or complex floating-point values, if an `atol > 0` is not specified, `rtol` defaults to the square root of [`eps`](@ref) of the type of `x` or `y`, whichever is bigger (least precise). This corresponds to requiring equality of about half of the significant digits. Otherwise, e.g. for integer arguments or if an `atol > 0` is supplied, `rtol` defaults to zero.

The `norm` keyword defaults to `abs` for numeric `(x,y)` and to `LinearAlgebra.norm` for arrays (where an alternative `norm` choice is sometimes useful). When `x` and `y` are arrays, if `norm(x-y)` is not finite (i.e. `±Inf` or `NaN`), the comparison falls back to checking whether all elements of `x` and `y` are approximately equal component-wise.

The binary operator `≈` is equivalent to `isapprox` with the default arguments, and `x ≉ y` is equivalent to `!isapprox(x,y)`.

Note that `x ≈ 0` (i.e., comparing to zero with the default tolerances) is equivalent to `x == 0` since the default `atol` is `0`.  In such cases, you should either supply an appropriate `atol` (or use `norm(x) ≤ atol`) or rearrange your code (e.g. use `x ≈ y` rather than `x - y ≈ 0`).   It is not possible to pick a nonzero `atol` automatically because it depends on the overall scaling (the "units") of your problem: for example, in `x - y ≈ 0`, `atol=1e-9` is an absurdly small tolerance if `x` is the [radius of the Earth](https://en.wikipedia.org/wiki/Earth_radius) in meters, but an absurdly large tolerance if `x` is the [radius of a Hydrogen atom](https://en.wikipedia.org/wiki/Bohr_radius) in meters.

!!! compat "Julia 1.6"
    Passing the `norm` keyword argument when comparing numeric (non-array) arguments requires Julia 1.6 or later.


# Examples

```jldoctest
julia> isapprox(0.1, 0.15; atol=0.05)
true

julia> isapprox(0.1, 0.15; rtol=0.34)
true

julia> isapprox(0.1, 0.15; rtol=0.33)
false

julia> 0.1 + 1e-10 ≈ 0.1
true

julia> 1e-10 ≈ 0
false

julia> isapprox(1e-10, 0, atol=1e-8)
true

julia> isapprox([10.0^9, 1.0], [10.0^9, 2.0]) # using `norm`
true
```

---

```
isapprox(x; kwargs...) / ≈(x; kwargs...)
```

Create a function that compares its argument to `x` using `≈`, i.e. a function equivalent to `y -> y ≈ x`.

The keyword arguments supported here are the same as those in the 2-argument `isapprox`.

!!! compat "Julia 1.5"
    This method requires Julia 1.5 or later.



We can now benchmark the C code directly from Julia:

In [30]:
c_bench = @benchmark sum($a)

BenchmarkTools.Trial: 172 samples with 1 evaluation.
 Range (min … max):  19.985 ms … 58.165 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     28.538 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   29.089 ms ±  6.064 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

               ▄█▄                                             
  ▄▁▄▆▅▆▅▄▅▃▄▅▄█████▅▅▃▃▃▄▃▄▃▃▁▁▁▃▃▁▁▁▁▃▁▁▁▁▁▁▄▁▁▃▁▁▁▃▁▁▁▁▁▁▃ ▃
  20 ms           Histogram: frequency by time        54.5 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.